# Grover's Algorithm

The **Grover's Search** quantum kata is a series of exercises designed
to get you familiar with Grover's search algorithm.

It covers the following topics:

* writing oracles for Grover's search,
* performing steps of the algorithm, and
* putting it all together: Grover's search algorithm.

*Reading material:*

* [This Microsoft Learn module](https://docs.microsoft.com/learn/modules/solve-graph-coloring-problems-grovers-search/) offers a different, visual explanation of Grover's algorithm.
* The tasks follow the explanation from *Quantum Computation and Quantum Information* by Nielsen and Chuang.
  In the 10th anniversary edition, this is section 6.1.2 on pages 248-251.
* A different explanation of Grover's algorithm can be found in 
  [this Wikipedia article](https://en.wikipedia.org/wiki/Grover%27s_algorithm).
* [An Introduction to Quantum Algorithms](https://people.cs.umass.edu/~strubell/doc/quantum_tutorial.pdf) by Emma Strubell, pages 20-24.
* [Lecture 4: Grover's Algorithm](https://www.cs.cmu.edu/~odonnell/quantum15/lecture04.pdf) by John Wright.
* Lectures [12](https://cs.uwaterloo.ca/~watrous/QC-notes/QC-notes.12.pdf) and [13](https://cs.uwaterloo.ca/~watrous/QC-notes/QC-notes.13.pdf) by John Watrous.
* [This page](http://davidbkemp.github.io/animated-qubits/grover.html) has an animated demonstration of Grover's algorithm for a simple case.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blanks (marked with the `// ...` comments)
with some Q# code that solves the task.  To verify your answer, run the cell with Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty;
harder ones are marked with asterisks.

## Part I. Oracles for Grover's Search


### Task 1.1. The $|11...1\rangle$ Oracle
**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Flip the state of the target qubit (i.e., apply an X gate to it)
if the query register is in the $|11...1\rangle$ state,
and leave it unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Examples:**

* If the query register is in state $|00...0\rangle$, leave the target qubit unchanged.

* If the query register is in state $|10...0\rangle$, leave the target qubit unchanged.

* If the query register is in state $|11...1\rangle$, flip the target qubit.

* If the query register is in state $\frac{1}{\sqrt{2}} \big(|00...0\rangle + |11...1\rangle \big)$, and the target is in state $|0\rangle$,
the joint state of the query register and the target qubit should be $\frac{1}{\sqrt{2}} \big(|00...00\rangle + |11...11\rangle \big)$.

In [1]:
//%kata T11_Oracle_AllOnes 

operation Oracle_AllOnes (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    Controlled X(queryRegister, target);
}

Oracle_AllOnes

### Task 1.2. The $|1010...\rangle$ Oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

  Flip the state of the target qubit if the query register is in the $|1010...\rangle$ state;
that is, the state with alternating 1 and 0 values, with any number of qubits in the register.
Leave the state of the target qubit unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Examples:**

 * If the register is in state $|0000000\rangle$, leave the target qubit unchanged.
 * If the register is in state $|10101\rangle$, flip the target qubit.

In [2]:
//%kata T12_Oracle_AlternatingBits 

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // within {U} apply {V}
    // Within-Apply makes a U sandwich!
    
    within {
        // Flip all of the odd index qubits before CC...CX and after
        for i in 1..2..Length(queryRegister) {
            X(queryRegister[i]);
        }
    } apply {
        Controlled X(queryRegister, target);
    }
    
    // Why do we have to flip and unflip?
    // Need to control on every bit to ensure it matches the pattern exactly
    // We want to do it coherently, and leave the queryRegister as it was before we started!
    
    // 10101T
    // 11111T
    // CCCCCX
    // 10101(XT)
}

Oracle_AlternatingBits

### Task 1.3. Arbitrary Bit Pattern Oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. A bit pattern of length N represented as `Bool[]`

**Goal:**

  Flip the state of the target qubit if the query register is in the state described by the given bit pattern
(`true` represents qubit state One, and `false` represents Zero).
Leave the state of the target qubit unchanged if the query register is in any other state.
Leave the query register in the same state it started in.

**Example:**

  If the bit pattern is `[true, false]`, you need to flip the target qubit if and only if the qubits are in the $|10\rangle$ state.

In [3]:
//%kata T13_Oracle_ArbitraryPattern 

operation Oracle_ArbitraryPattern (queryRegister : Qubit[], target : Qubit, pattern : Bool[]) : Unit is Adj {
    // ControlledOnBitString takes a bool array 
    // and an operation to apply to the target
    // if the query register matches the pattern.
    (ControlledOnBitString(pattern, X))(queryRegister, target);
}

Oracle_ArbitraryPattern

### Task 1.4. Oracle Converter

**Input:**

A marking oracle: an oracle that takes a register and a target qubit and
flips the target qubit if the register satisfies a certain condition.

**Output:**

A phase-flipping oracle: an oracle that takes a register and
flips the phase of the register if it satisfies this condition.

> Grover's algorithm relies on the search condition implemented as a phase-flipping oracle,
but it is often easier to write a marking oracle for a given condition. This transformation
allows to convert one type of oracle into the other. The transformation is described in the 
[Wikipedia article on Grover's algorithm](https://en.wikipedia.org/wiki/Grover%27s_algorithm), in the section "Description of ${U_\omega}$".

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
    Remember that you can define auxiliary operations. To do that, you'll need to create an extra code cell for each new operation and execute it before returning to this cell.
</details>

In [4]:
//%kata T14_OracleConverter 

operation OracleConverterHelper (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    // Off to Wikipedia!
    
    // Well anyways, the plan is to use |-> as the target qubit
    // |x, -> == |x>(1/√2 (|0> - |1>))
    // = IX => |x>, 1/√2 (|1> - |0>) == |x>(-|->) == -|x>|->
    // We happily push this global phase out onto the x, leaving the original |-> state!
    
    // Declare that we're using a fresh qubit, which means we're responsible for clearing it
    // before it goes out of scope at the end of this block
    
    // Request a qubit
    use target = Qubit();
    within {
        // Put the target into the |-⟩ state
        X(target);  // |1>
        H(target);  // |->
    } apply {
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
        // (phase kickback trick)
        markingOracle(register, target);  // We got this from above with the CC...CX oracle
    }
}

OracleConverterHelper

In [5]:
// Functions need a return statement.

// Operations: work on quantum registers
// Functions: classical subroutines

function OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    return OracleConverterHelper(markingOracle, _);
}

OracleConverter

## Part II. The Grover Iteration

### Task 2.1. The Hadamard Transform

**Input:** A register of N qubits in an arbitrary state

**Goal:** Apply the Hadamard transform to each of the qubits in the register.

> If the register started in the $|0...0\rangle$ state, this operation
will prepare an equal superposition of all $2^{N}$ basis states.

In [6]:
//%kata T21_HadamardTransform 

operation HadamardTransform (register : Qubit[]) : Unit is Adj {
    // Cheeky library op to make this really easy
    ApplyToEachA(H, register);
    
    // Long winded way to say it
    // for (qubit in register) {
    //    H(qubit);
    // }
}

HadamardTransform

### Task 2.2. Conditional Phase Flip

**Input:**  A register of N qubits in an arbitrary state.

**Goal:**  Flip the sign of the state of the register if it is not in the $|0...0\rangle$ state.

**Examples:**

 * If the register is in state $|0...0\rangle$, leave it unchanged.

 * If the register is in any other basis state, multiply its phase by -1.

> This operation implements operator $2|0...0\rangle\langle0...0| - I$ $ = \left(\begin{matrix}1&0&...&0\\0&-1&...&0\\\vdots&\vdots&\ddots&\vdots\\0&0&...&-1\end{matrix}\right) $

<br/>
<details>
  <summary><b>Hint #1</b></summary>
    Note that quantum states are defined up to a global phase.
    Thus the state obtained as a result of this operation is equivalent to the state obtained by flipping the sign of only the $|0...0\rangle$ basis state (those states differ by a global phase $-1$).<br>
    $$-\big(2|0...0\rangle\langle0...0| - I\big) = I - 2|0...0\rangle\langle0...0| = \left(\begin{matrix}-1&0&...&0\\0&1&...&0\\\vdots&\vdots&\ddots&\vdots\\0&0&...&1\end{matrix}\right) $$<br>
 
It doesn't matter for Grover's search algorithm itself, since the global phase is not observable, but can have side effects when used as part of other algorithms.
> See the extended discussion in this [Quantum Computing SE question](https://quantumcomputing.stackexchange.com/questions/5973/counting-in-q-number-of-solutions/6446#6446)<br>
    </details>
<br/>

<details>
  <summary><b>Hint #2</b></summary>
    Consider the Controlled Z gate, applied with most of the qubits as control and the last qubit as target:
    $\text{Controlled Z}(|s_0 s_1 \ldots s_{n-2}\rangle, |s_{n-1}\rangle)$ leaves all basis states except $|1...11\rangle$ unchanged, and adds a $-1$ phase to that state: $|1...11\rangle \rightarrow -|1...11\rangle$ (remember that $Z|0\rangle = |0\rangle$ and $Z|1\rangle = -|1\rangle$). 
    You need to modify it to add the $-1$ phase to only the $|0...00\rangle$ state instead.
    <br/><br/>
    Alternatively, you can use the same trick as in the oracle converter task.<br>
</details>
<br>

<details>
  <summary><b>Hint #3</b></summary>
  You can use the <a href="https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.r">R gate</a> to correct the global phase.
</details>

In [7]:
//%kata T22_ConditionalPhaseFlip 

// Include for pi
open Microsoft.Quantum.Math;

operation ConditionalPhaseFlip (register : Qubit[]) : Unit is Adj {
    // Initialization is to False
    let allZerosOracle = Oracle_ArbitraryPattern(_, _, new Bool[Length(register)]);
    
    // Get a phase flip oracle for it from our function
    let phaseFlipOracle = OracleConverter(allZerosOracle);
    
    // Actually apply the phase oracle
    phaseFlipOracle(register);
    
    // Global phase
    R(PauliI, 2.0 * PI(), register[0]);
}

ConditionalPhaseFlip

Why did we need a global phase?

Our phase oracle implements operator $I - 2|0...0\rangle\langle0...0|$ $ = \left(\begin{matrix}-1&0&...&0\\0&1&...&0\\\vdots&\vdots&\ddots&\vdots\\0&0&...&1\end{matrix}\right) $,

and $I - 2|0...0\rangle\langle0...0| = -(2|0...0\rangle\langle0...0| - I)$.

### Task 2.3. The Grover Iteration

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. A phase-flipping oracle that takes an N-qubit register and flips
     the phase of the state if the register is in the desired state.

**Goal:**  Perform one Grover iteration.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
    A Grover iteration consists of 4 steps:
    <ol>
    <li>Apply the Oracle</li>
    <li>Apply the Hadamard transform</li>
    <li>Perform a conditional phase shift</li>
    <li>Apply the Hadamard transform again</li>
    </ol>
</details>

In [8]:
//%kata T23_GroverIteration 

// Plan for one iteration:
// - Query the oracle
// - Phase flip about the uniform superposition

operation GroverIteration (register : Qubit[], oracle : (Qubit[] => Unit is Adj)) : Unit is Adj {
    oracle(register);
    
    // Flipping about all 0 in the Hadamard basis
    // exactly means flipping about all +, or the uniform superposition
    // in the computational basis.
    within {
        HadamardTransform(register);
    } apply {
        ConditionalPhaseFlip(register);
    }
}

GroverIteration

## Part III. Putting It All Together: Grover's Search Algorithm

### Task 3.1. Grover's Search

**Inputs:**

  1. N qubits in the $|0...0\rangle$ state.

  2. A marking oracle.

  3. The number of Grover iterations to perform.

**Goal:** Use Grover's algorithm to leave the register in the state that is marked by the oracle as the answer (with high probability).

> The number of iterations is passed as a parameter because it is defined by the nature of the problem
and is easier to configure/calculate outside the search algorithm itself (for example, in the classical driver).

In [9]:
//%kata T31_GroversSearch 

operation GroversSearch (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    let phaseFlipOracle = OracleConverter(oracle);
    
    // Start things off in the uniform superposition
    HadamardTransform(register);
    
    for iter in 1..iterations {
        GroverIteration(register, phaseFlipOracle);
    }
}

GroversSearch

### Task 3.2. Using Grover's Search

**Goal:**   Use your implementation of Grover's Algorithm from Task 3.1 and the oracles from part 1
  to find the marked elements of the search space. This task is not covered by a test and allows you to experiment with running the algorithm.
  
> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details closed>
  <summary><b>Hint #1</b></summary>
    To check whether the algorithm found the correct answer (i.e., an answer marked as 1 by the oracle),
    you can apply the oracle once more to the register after you've measured it and an ancilla qubit,
    which will calculate the function of the answer found by the algorithm.
</details>
<br/>
<details closed>
  <summary><b>Hint #2</b></summary>
    Experiment with the number of iterations to see how it affects
    the probability of the algorithm finding the correct answer.
</details>
<br/>
<details closed>
  <summary><b>Hint #3</b></summary>
    You can use the Message function to output the results.
</details>

In [ ]:
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;

operation Run_GroversSearch_Algorithm () : Unit {
    let num_qubits = 30;
    
    use register = Qubit[num_qubits];
    
    // Give our function a marking oracle
    
    let pattern1 = [true, false, false, false];
    let pattern2 = [false, true, false, false];
    let pattern4 = [false, false, true, false];
    let pattern8 = [false, false, false, true];
    let pattern15 = [true, true, true, true, false, false];
    
    let pattern_0_30 = new Bool[30];

    
    let markingOracle = Oracle_ArbitraryPattern(_, _, pattern_0_50);
    
    // 3x this?
    let iterations = Floor(PI()/4.0 * Sqrt(IntAsDouble(PowI(2, num_qubits))));
    
    GroversSearch(register, markingOracle, iterations);
    
    DumpRegister((), register);
        
    ResetAll(register);
}

In [15]:
%simulate Run_GroversSearch_Algorithm

|0⟩	-0.005135846372143584 + -2.5158391282232665E-18𝑖
|1⟩	-0.005135846372143435 + -2.515839128223256E-18𝑖
|2⟩	-0.005135846372143525 + -2.515839128223258E-18𝑖
|3⟩	-0.005135846372143536 + -2.5158391282232638E-18𝑖
|4⟩	-0.005135846372143584 + -2.515839128223236E-18𝑖
|5⟩	-0.0051358463721434345 + -2.515839128223195E-18𝑖
|6⟩	-0.005135846372143525 + -2.515839128223236E-18𝑖
|7⟩	-0.00513584637214353 + -2.5158391282232572E-18𝑖
|8⟩	-0.005135846372143584 + -2.5158391282232665E-18𝑖
|9⟩	-0.0051358463721434345 + -2.515839128223256E-18𝑖
|10⟩	-0.005135846372143525 + -2.515839128223258E-18𝑖
|11⟩	-0.00513584637214353 + -2.515839128223266E-18𝑖
|12⟩	-0.005135846372143582 + -2.515839128223237E-18𝑖
|13⟩	-0.005135846372143478 + -2.515839128223193E-18𝑖
|14⟩	-0.00513584637214354 + -2.5158391282232222E-18𝑖
|15⟩	0.9995910741614762 + 4.896584037712466E-16𝑖
|16⟩	-0.005135846372143525 + -2.515839128223248E-18𝑖
|17⟩	-0.005135846372143525 + -2.515839128223249E-18𝑖
|18⟩	-0.005135846372143525 + -2.515839128223248E-18𝑖
|19⟩	-0.005135846372143514 + -2.5158391282232445E-18𝑖
|20⟩	-0.005135846372143525 + -2.515839128223248E-18𝑖
|21⟩	-0.005135846372143525 + -2.5158391282232445E-18𝑖
|22⟩	-0.005135846372143525 + -2.515839128223248E-18𝑖
|23⟩	-0.005135846372143514 + -2.5158391282232576E-18𝑖
|24⟩	-0.005135846372143525 + -2.515839128223248E-18𝑖
|25⟩	-0.005135846372143525 + -2.515839128223249E-18𝑖
|26⟩	-0.005135846372143525 + -2.515839128223248E-18𝑖
|27⟩	-0.005135846372143514 + -2.5158391282232445E-18𝑖
|28⟩	-0.0051358463721435255 + -2.5158391282232472E-18𝑖
|29⟩	-0.005135846372143523 + -2.5158391282232453E-18𝑖
|30⟩	-0.005135846372143548 + -2.515839128223244E-18𝑖
|31⟩	-0.00513584637214353 + -2.515839128223227E-18𝑖

()

In [ ]:
%estimate Run_GroversSearch_Algorithm